In [654]:
import re
import pandas as pd
import numpy as np
# from mlutils import dataset, connector
import scipy.stats
from scipy.stats import *
from sklearn.preprocessing import StandardScaler
import math
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from pandasql import sqldf
import datetime
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from math import sqrt
warnings.filterwarnings('ignore')
%matplotlib inline

In [573]:
df_exercise = pd.read_csv('Exercise_data.csv')

In [574]:
df_exercise.head()

,Date,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories
0,01/01/20,"1,528",0,0.0,0,"1,440",0,0,0,0
1,02/01/20,"1,528",0,0.0,0,"1,440",0,0,0,0
2,03/01/20,"1,528",0,0.0,0,"1,440",0,0,0,0
3,04/01/20,"1,528",0,0.0,0,"1,440",0,0,0,0
4,05/01/20,"1,528",0,0.0,0,"1,440",0,0,0,0


In [575]:
df_exercise.dtypes

Date                       object
Calories Burned            object
Steps                      object
Distance                  float64
Floors                      int64
Minutes Sedentary          object
Minutes Lightly Active      int64
Minutes Fairly Active       int64
Minutes Very Active         int64
Activity Calories          object
dtype: object

In [576]:
df_exercise['Steps'] = df_exercise['Steps'].apply(lambda a : int(a.replace(',', '')))

In [577]:
df_exercise['Calories Burned'] = df_exercise['Calories Burned'].apply(lambda a : int(a.replace(',', '')))

In [578]:
df_exercise['Activity Calories'] = df_exercise['Minutes Sedentary'].apply(lambda a : int(a.replace(',', '')))

In [579]:
df_exercise['Minutes Sedentary'] = df_exercise['Minutes Sedentary'].apply(lambda a : int(a.replace(',', '')))

In [580]:
df_exercise['Date']= pd.to_datetime(df_exercise['Date']) 

In [581]:
df_exercise.head()

,Date,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories
0,2020-01-01,1528,0,0.0,0,1440,0,0,0,1440
1,2020-02-01,1528,0,0.0,0,1440,0,0,0,1440
2,2020-03-01,1528,0,0.0,0,1440,0,0,0,1440
3,2020-04-01,1528,0,0.0,0,1440,0,0,0,1440
4,2020-05-01,1528,0,0.0,0,1440,0,0,0,1440


In [582]:
df_exercise.dtypes

Date                      datetime64[ns]
Calories Burned                    int64
Steps                              int64
Distance                         float64
Floors                             int64
Minutes Sedentary                  int64
Minutes Lightly Active             int64
Minutes Fairly Active              int64
Minutes Very Active                int64
Activity Calories                  int64
dtype: object

In [583]:
df_exercise.columns

Index(['Date', 'Calories Burned', 'Steps', 'Distance', 'Floors',
       'Minutes Sedentary', 'Minutes Lightly Active', 'Minutes Fairly Active',
       'Minutes Very Active', 'Activity Calories'],
      dtype='object')

In [584]:
df_exercise.columns = ['Date', 'CaloriesBurned', 'Steps', 'Distance', 'Floors',
       'MinutesSedentary', 'MinutesLightlyActive', 'MinutesFairlyActive',
       'MinutesVeryActive', 'ActivityCalories']

In [585]:
cols = ['Date', 'CaloriesBurned', 'Steps', 'Distance', 
       'MinutesSedentary', 'MinutesLightlyActive', 'MinutesFairlyActive',
       'MinutesVeryActive']

df_exercise = df_exercise[cols]

In [586]:
day_name= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
df_exercise['Day'] = df_exercise['Date'].apply(lambda x: day_name[x.weekday()])

In [587]:
df_exercise.head()

,Date,CaloriesBurned,Steps,Distance,MinutesSedentary,MinutesLightlyActive,MinutesFairlyActive,MinutesVeryActive,Day
0,2020-01-01,1528,0,0.0,1440,0,0,0,Wednesday
1,2020-02-01,1528,0,0.0,1440,0,0,0,Saturday
2,2020-03-01,1528,0,0.0,1440,0,0,0,Sunday
3,2020-04-01,1528,0,0.0,1440,0,0,0,Wednesday
4,2020-05-01,1528,0,0.0,1440,0,0,0,Friday


In [588]:
df_exercise.sort_values("Date", axis = 0, ascending = True, 
                 inplace = True) 

In [589]:
df_exercise.head()

,Date,CaloriesBurned,Steps,Distance,MinutesSedentary,MinutesLightlyActive,MinutesFairlyActive,MinutesVeryActive,Day
0,2020-01-01,1528,0,0.00,1440,0,0,0,Wednesday
31,2020-01-02,1705,1999,1.39,939,73,0,0,Thursday
60,2020-01-03,2764,16048,11.16,613,337,54,42,Friday
91,2020-01-04,2136,7058,4.91,818,209,36,2,Saturday
12,2020-01-13,1528,0,0.00,1440,0,0,0,Monday


In [590]:
df_exercise_preLockdown = df_exercise[(df_exercise['Date'] >= '2020-02-13') & (df_exercise['Date'] <= '2020-03-15')
                                      & (~(df_exercise['Day'].isin(['Saturday','Sunday'])) ) & (df_exercise['Steps']!=0)]

In [591]:
df_exercise_preLockdown.head()

,Date,CaloriesBurned,Steps,Distance,MinutesSedentary,MinutesLightlyActive,MinutesFairlyActive,MinutesVeryActive,Day
43,2020-02-13,2755,16459,11.44,740,305,46,46,Thursday
44,2020-02-14,2896,19163,13.43,495,342,70,21,Friday
47,2020-02-17,2229,9037,6.29,780,197,30,25,Monday
48,2020-02-18,2437,11677,8.24,759,295,16,14,Tuesday
49,2020-02-19,2330,9962,6.92,1133,281,16,10,Wednesday


In [592]:
df_exercise_preLockdown.shape

(16, 9)

In [593]:
df_exercise_postLockdown = df_exercise[(df_exercise['Date'] >= '2020-03-16') &
                                       (~(df_exercise['Day'].isin(['Saturday','Sunday'])) ) & (df_exercise['Steps']!=0)]

In [594]:
df_exercise_postLockdown.head()

,Date,CaloriesBurned,Steps,Distance,MinutesSedentary,MinutesLightlyActive,MinutesFairlyActive,MinutesVeryActive,Day
75,2020-03-16,2191,7389,5.14,723,274,9,0,Monday
76,2020-03-17,2403,10609,7.44,751,297,30,10,Tuesday
77,2020-03-18,2235,7958,5.54,864,264,30,2,Wednesday
78,2020-03-19,2007,5205,3.62,794,199,8,1,Thursday
79,2020-03-20,2159,6908,4.89,796,242,17,6,Friday


In [595]:
df_exercise_postLockdown.shape

(28, 9)

In [596]:
df_exercise = pd.concat([df_exercise_preLockdown,df_exercise_postLockdown])

In [597]:
df_exercise.dtypes

Date                    datetime64[ns]
CaloriesBurned                   int64
Steps                            int64
Distance                       float64
MinutesSedentary                 int64
MinutesLightlyActive             int64
MinutesFairlyActive              int64
MinutesVeryActive                int64
Day                             object
dtype: object

In [598]:
df_sleep = pd.read_csv('Sleep_data.csv')

In [599]:
df_sleep.head()

,StartTime,EndTime,MinutesAsleep,MinutesAwake,NumberOfAwakenings,TimeInBed,MinutesRemSleep,MinutesLightSleep,MinutesDeepSleep
0,"Sunday, 19 January 2020","Sunday, 19 January 2020",375,13,6,404,NaN,NaN,NaN
1,"Monday, 20 January 2020","Monday, 20 January 2020",345,44,2,389,NaN,NaN,NaN
2,"Tuesday, 21 January 2020","Tuesday, 21 January 2020",418,53,5,475,NaN,NaN,NaN
3,"Wednesday, 22 January 2020","Wednesday, 22 January 2020",352,39,3,391,NaN,NaN,NaN
4,"Thursday, 23 January 2020","Thursday, 23 January 2020",376,45,2,427,NaN,NaN,NaN


In [600]:
df_sleep.dtypes

StartTime              object
EndTime                object
MinutesAsleep           int64
MinutesAwake            int64
NumberOfAwakenings      int64
TimeInBed               int64
MinutesRemSleep       float64
MinutesLightSleep     float64
MinutesDeepSleep      float64
dtype: object

In [601]:
df_sleep['StartTime']= pd.to_datetime(df_sleep['StartTime']).dt.strftime('%Y-%m-%d')
df_sleep['EndTime']= pd.to_datetime(df_sleep['EndTime']).dt.strftime('%Y-%m-%d')

In [602]:
df_sleep.head()

,StartTime,EndTime,MinutesAsleep,MinutesAwake,NumberOfAwakenings,TimeInBed,MinutesRemSleep,MinutesLightSleep,MinutesDeepSleep
0,2020-01-19,2020-01-19,375,13,6,404,NaN,NaN,NaN
1,2020-01-20,2020-01-20,345,44,2,389,NaN,NaN,NaN
2,2020-01-21,2020-01-21,418,53,5,475,NaN,NaN,NaN
3,2020-01-22,2020-01-22,352,39,3,391,NaN,NaN,NaN
4,2020-01-23,2020-01-23,376,45,2,427,NaN,NaN,NaN


In [603]:
df_sleep.columns

Index(['StartTime', 'EndTime', 'MinutesAsleep', 'MinutesAwake',
       'NumberOfAwakenings', 'TimeInBed', 'MinutesRemSleep',
       'MinutesLightSleep', 'MinutesDeepSleep'],
      dtype='object')

In [604]:
cols = ['StartTime','EndTime', 'MinutesAsleep', 'MinutesAwake',
       'NumberOfAwakenings', 'TimeInBed']

In [605]:
df_sleep = df_sleep[cols]

In [606]:
df_sleep.head()

,StartTime,EndTime,MinutesAsleep,MinutesAwake,NumberOfAwakenings,TimeInBed
0,2020-01-19,2020-01-19,375,13,6,404
1,2020-01-20,2020-01-20,345,44,2,389
2,2020-01-21,2020-01-21,418,53,5,475
3,2020-01-22,2020-01-22,352,39,3,391
4,2020-01-23,2020-01-23,376,45,2,427


In [607]:
df_sleep.shape

(98, 6)

In [608]:
query = """
            select 
                EndTime as Date, 
                sum(MinutesAsleep) as MinutesAsleep,
                sum(MinutesAwake) as MinutesAwake,
                sum(NumberOfAwakenings) as NumberOfAwakenings,
                sum(TimeinBed) as TimeInBed
            from 
                df_sleep
            group by 
                EndTime
            order by
                Date
        """

df_sleep_agg = sqldf(query,globals())

In [609]:
df_sleep_agg.shape

(78, 5)

In [610]:
df_sleep_agg.head()

,Date,MinutesAsleep,MinutesAwake,NumberOfAwakenings,TimeInBed
0,2020-01-19,375,13,6,404
1,2020-01-20,345,44,2,389
2,2020-01-21,418,53,5,475
3,2020-01-22,352,39,3,391
4,2020-01-23,376,45,2,427


In [611]:
df_sleep_agg.tail()

,Date,MinutesAsleep,MinutesAwake,NumberOfAwakenings,TimeInBed
73,2020-04-03,305,27,1,332
74,2020-04-04,411,45,6,456
75,2020-04-05,323,40,2,364
76,2020-04-06,361,47,2,408
77,2020-04-07,344,39,3,383


In [612]:
df_sleep_agg.Date = pd.to_datetime(df_sleep_agg['Date']) 

In [613]:
df_sleep_agg.dtypes

Date                  datetime64[ns]
MinutesAsleep                  int64
MinutesAwake                   int64
NumberOfAwakenings             int64
TimeInBed                      int64
dtype: object

In [614]:
df_exercise.dtypes

Date                    datetime64[ns]
CaloriesBurned                   int64
Steps                            int64
Distance                       float64
MinutesSedentary                 int64
MinutesLightlyActive             int64
MinutesFairlyActive              int64
MinutesVeryActive                int64
Day                             object
dtype: object

In [615]:
df_exercise['incremented_date']= df_exercise.Date + datetime.timedelta(days=1)

In [616]:
df_exercise.head()

,Date,CaloriesBurned,Steps,Distance,MinutesSedentary,MinutesLightlyActive,MinutesFairlyActive,MinutesVeryActive,Day,incremented_date
43,2020-02-13,2755,16459,11.44,740,305,46,46,Thursday,2020-02-14
44,2020-02-14,2896,19163,13.43,495,342,70,21,Friday,2020-02-15
47,2020-02-17,2229,9037,6.29,780,197,30,25,Monday,2020-02-18
48,2020-02-18,2437,11677,8.24,759,295,16,14,Tuesday,2020-02-19
49,2020-02-19,2330,9962,6.92,1133,281,16,10,Wednesday,2020-02-20


In [617]:
query = """
            select 
                a.*,
                b.MinutesAsleep
            from
                df_exercise a
            join 
                df_sleep_agg b
            on
                a.incremented_date = b.Date
            order by
                a.Date
                
        """
df_join = sqldf(query,globals())

In [618]:
df_join.head()

,Date,CaloriesBurned,Steps,Distance,MinutesSedentary,MinutesLightlyActive,MinutesFairlyActive,MinutesVeryActive,Day,incremented_date,MinutesAsleep
0,2020-02-13 00:00:00.000000,2755,16459,11.44,740,305,46,46,Thursday,2020-02-14 00:00:00.000000,461
1,2020-02-14 00:00:00.000000,2896,19163,13.43,495,342,70,21,Friday,2020-02-15 00:00:00.000000,318
2,2020-02-17 00:00:00.000000,2229,9037,6.29,780,197,30,25,Monday,2020-02-18 00:00:00.000000,323
3,2020-02-19 00:00:00.000000,2330,9962,6.92,1133,281,16,10,Wednesday,2020-02-20 00:00:00.000000,133
4,2020-02-20 00:00:00.000000,2384,10723,7.46,971,311,16,0,Thursday,2020-02-21 00:00:00.000000,491


In [697]:
df_join.Date = pd.to_datetime(df_join['Date']) 
df_join.incremented_date = pd.to_datetime(df_join['incremented_date']) 

In [620]:
df_join.dtypes

Date                    datetime64[ns]
CaloriesBurned                   int64
Steps                            int64
Distance                       float64
MinutesSedentary                 int64
MinutesLightlyActive             int64
MinutesFairlyActive              int64
MinutesVeryActive                int64
Day                             object
incremented_date                object
MinutesAsleep                    int64
dtype: object

In [621]:
df_join.head()

,Date,CaloriesBurned,Steps,Distance,MinutesSedentary,MinutesLightlyActive,MinutesFairlyActive,MinutesVeryActive,Day,incremented_date,MinutesAsleep
0,2020-02-13,2755,16459,11.44,740,305,46,46,Thursday,2020-02-14 00:00:00.000000,461
1,2020-02-14,2896,19163,13.43,495,342,70,21,Friday,2020-02-15 00:00:00.000000,318
2,2020-02-17,2229,9037,6.29,780,197,30,25,Monday,2020-02-18 00:00:00.000000,323
3,2020-02-19,2330,9962,6.92,1133,281,16,10,Wednesday,2020-02-20 00:00:00.000000,133
4,2020-02-20,2384,10723,7.46,971,311,16,0,Thursday,2020-02-21 00:00:00.000000,491


In [696]:
df_join.tail()

,Date,CaloriesBurned,Steps,Distance,MinutesSedentary,MinutesLightlyActive,MinutesFairlyActive,MinutesVeryActive,Day,incremented_date,MinutesAsleep
23,2020-03-26,1921,4487,3.04,838,137,22,4,Thursday,2020-03-27 00:00:00.000000,288
24,2020-03-30,2274,8603,5.98,965,283,20,5,Monday,2020-03-31 00:00:00.000000,347
25,2020-03-31,2076,6376,4.44,838,193,31,2,Tuesday,2020-04-01 00:00:00.000000,366
26,2020-04-02,1935,4957,3.44,824,146,4,4,Thursday,2020-04-03 00:00:00.000000,305
27,2020-04-03,2688,16671,11.59,633,250,43,47,Friday,2020-04-04 00:00:00.000000,411


In [698]:
df_join[['Date', 'CaloriesBurned', 'Steps', 'Distance', 'MinutesAsleep','incremented_date']].head()

,Date,CaloriesBurned,Steps,Distance,MinutesAsleep,incremented_date
0,2020-02-13,2755,16459,11.44,461,2020-02-14
1,2020-02-14,2896,19163,13.43,318,2020-02-15
2,2020-02-17,2229,9037,6.29,323,2020-02-18
3,2020-02-19,2330,9962,6.92,133,2020-02-20
4,2020-02-20,2384,10723,7.46,491,2020-02-21


In [623]:
df_join.columns

Index(['Date', 'CaloriesBurned', 'Steps', 'Distance', 'MinutesSedentary',
       'MinutesLightlyActive', 'MinutesFairlyActive', 'MinutesVeryActive',
       'Day', 'incremented_date', 'MinutesAsleep'],
      dtype='object')

In [624]:
def to_proper_format(value):
    if(type(value)!= np.ndarray):
        hours = int(value/60)
        minutes = math.ceil(value % 60)
        if(minutes == 60):
            minutes = 0
            hours+=1
        if hours == 0:
            return ["{} minutes".format(minutes)]
        return ["{} hours {} minutes".format(hours,minutes)]
    else:
        for v in value:
            if(len(value) == 1):
                return to_proper_format(v)
            else:
                return to_proper_format(v) + to_proper_format(value[1:])

In [625]:
cols = ['CaloriesBurned']
clf = LinearRegression()
clf.fit(df_join[cols],df_join['MinutesAsleep'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [626]:
clf.coef_

array([0.04415075])

In [627]:
clf.intercept_

253.7704133558832

In [628]:
clf.predict(df_join[cols])

array([375.4057427 , 381.63099912, 352.1824457 , 356.64167193,
       359.02581268, 358.62845589, 367.32615458, 361.71900872,
       358.45185287, 368.42992345, 384.85400421, 348.87113909,
       375.4057427 , 351.03452607, 347.72321947, 350.50471702,
       359.86467702, 352.44735023, 342.38097814, 349.09189286,
       352.97715928, 346.17794305, 340.39419418, 338.58401323,
       354.16922966, 345.42738022, 339.2021238 , 372.44764213])

In [629]:
res = clf.predict([[1000],[2000],[3000]])
res,type(res)

(array([297.92116811, 342.07192286, 386.22267761]), numpy.ndarray)

In [707]:
data = [[2100],[2200],[2500],[2600],[2700],[2969],[3000]]
res = to_proper_format(clf.predict(data))
res
for index,value in enumerate(data):
    print('Calories Burnt: {} Kcalorie\nAmount of Ideal sleep needed for your body-type: {}\n\n'.format(value[0],res[index]))

Calories Burnt: 2100 Kcalorie
Amount of Ideal sleep needed for your body-type: 5 hours 47 minutes


Calories Burnt: 2200 Kcalorie
Amount of Ideal sleep needed for your body-type: 5 hours 51 minutes


Calories Burnt: 2500 Kcalorie
Amount of Ideal sleep needed for your body-type: 6 hours 5 minutes


Calories Burnt: 2600 Kcalorie
Amount of Ideal sleep needed for your body-type: 6 hours 9 minutes


Calories Burnt: 2700 Kcalorie
Amount of Ideal sleep needed for your body-type: 6 hours 13 minutes


Calories Burnt: 2969 Kcalorie
Amount of Ideal sleep needed for your body-type: 6 hours 25 minutes


Calories Burnt: 3000 Kcalorie
Amount of Ideal sleep needed for your body-type: 6 hours 27 minutes




In [687]:
pre_lst = df_exercise_preLockdown['CaloriesBurned'].to_list()
pre_lst.sort()
pre_lst[:5],pre_lst[-5:]

([2128, 2154, 2203, 2229, 2330], [2597, 2755, 2755, 2896, 2969])

In [686]:
post_lst = df_exercise_postLockdown['CaloriesBurned'].to_list()
post_lst.sort()
post_lst[:5],post_lst[-5:]

([1864, 1921, 1935, 1959, 1962], [2688, 2729, 2777, 2827, 3224])

In [633]:
df_join

,Date,CaloriesBurned,Steps,Distance,MinutesSedentary,MinutesLightlyActive,MinutesFairlyActive,MinutesVeryActive,Day,incremented_date,MinutesAsleep
0,2020-02-13,2755,16459,11.44,740,305,46,46,Thursday,2020-02-14 00:00:00.000000,461
1,2020-02-14,2896,19163,13.43,495,342,70,21,Friday,2020-02-15 00:00:00.000000,318
2,2020-02-17,2229,9037,6.29,780,197,30,25,Monday,2020-02-18 00:00:00.000000,323
3,2020-02-19,2330,9962,6.92,1133,281,16,10,Wednesday,2020-02-20 00:00:00.000000,133
4,2020-02-20,2384,10723,7.46,971,311,16,0,Thursday,2020-02-21 00:00:00.000000,491
5,2020-02-21,2375,10849,7.55,612,288,22,4,Friday,2020-02-22 00:00:00.000000,461
6,2020-02-24,2572,13822,9.61,830,316,25,15,Monday,2020-02-25 00:00:00.000000,429
7,2020-02-25,2445,12152,8.45,682,257,37,26,Tuesday,2020-02-26 00:00:00.000000,232
8,2020-02-26,2371,11322,7.87,920,217,32,31,Wednesday,2020-02-27 00:00:00.000000,304
9,2020-02-27,2597,13894,9.65,739,315,36,30,Thursday,2020-02-28 00:00:00.000000,295


In [648]:
clf2 = LinearRegression(fit_intercept=True)
clf2.fit(df_join[['Steps']],df_join['CaloriesBurned'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [653]:
clf2.intercept_,clf2.coef_

(1682.9788386479186, array([0.06376429]))

In [655]:
rms = sqrt(mean_squared_error(df_join['CaloriesBurned'], clf2.predict(df_join[['Steps']])))

In [656]:
rms

33.13859160249959

<h1>100 steps means 6.37 calories<br>
1 km = 1338 steps</h1>


In [670]:
def burn(calorie):
    return "{} steps or {} kms".format(int(100/6.3764 * calorie),round(((100/6.3764 * calorie)/1338.0),2))

In [684]:
burn(450)

'7057 steps or 5.27 kms'

In [685]:
450 * 0.04415

19.8675

In [681]:
253/60

4.216666666666667

In [688]:
(2896-2755)/2755,(19163-16459)/16459

(0.05117967332123412, 0.1642870162221277)

In [692]:
clf2.intercept_+ clf2.coef_ * 2924

array([1869.42561669])